### Setup and Library Imports

In [1]:
!pip install openai==0.28.0 comet_llm pandas numpy --quiet

In [2]:
import comet_llm
import openai
import json
import pandas as pd
import numpy as np
from google.colab import userdata

# API configuration
openai.api_key = userdata.get('OPENAI_API_KEY')
COMET_API_KEY = userdata.get('COMET_API_KEY')
COMET_WORKSPACE = userdata.get('COMET_WORKSPACE')

# Initialize Comet for logging
comet_llm.init(project="my_clickbait_detection_project", api_key=COMET_API_KEY)

# Load Training Data
with open('/content/data/headlines_dataset.txt', 'r') as file:
    training_data = json.load(file)
training_df = pd.DataFrame(training_data)

# Load Evaluation Data
with open('/content/data/evaluation_data.txt', 'r') as file:
    evaluation_data = json.load(file)
evaluation_df = pd.DataFrame(evaluation_data)


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


### Data Preparation and Templates.

In [3]:
# Preparing data for processing
training_headlines = training_df['headline'].tolist()
training_labels = training_df['category'].tolist()
evaluation_headlines = evaluation_df['headline'].tolist()
evaluation_labels = evaluation_df['tag'].tolist()


# Zero-shot prompt template
zero_shot_template = """
A headline is considered 'clickbait' if it:
1. Uses sensationalist or exaggerated language.
2. Teases the reader about a topic without providing substantial information.
3. Uses misleading statements or half-truths.
4. Provokes curiosity or emotional reaction to entice clicks.

Read the following headline and classify it as 'Clickbait' or 'Legit':

Headline: {headline}

"""

# Few-shot prompt template
few_shot_examples = training_df.sample(6).to_dict(orient='records')
few_shot_template = """
A headline is considered 'clickbait' if it:
1. Uses sensationalist or exaggerated language.
2. Teases the reader about a topic without providing substantial information.
3. Uses misleading statements or half-truths.
4. Provokes curiosity or emotional reaction to entice clicks.

Examples:
""" + "\n".join([f"Headline: {example['headline']}\nClassification: {example['category']}" for example in few_shot_examples]) + """

Read the following headline and classify it as 'Clickbait' or 'Legit':

Headline: {headline}

"""

# CoT prompt template - revised for clearer responses
cot_template = """
Your task is to classify the following headline as either 'Clickbait' or 'Legit'.

Headline: {headline}

Considerations for 'Clickbait':
- Uses sensationalist or exaggerated language.
- Teases the reader without providing substantial information.
- Contains misleading statements or half-truths.
- Provokes curiosity or emotional reaction to entice clicks.

Considerations for 'Legit':
- Provides clear and straightforward information.
- Lacks sensationalism or provocative elements.

Classification:
"""


### Functions for LLM Interactions and Headline Processing.

In [4]:
# Function to get a response from the GPT-3.5 Turbo model
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=300):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

def moderate_content(headline):
    response = openai.Moderation.create(
        input=headline,
        model="text-moderation-stable"
    )
    return response

def rewrite_headline(headline):
    rewrite_prompt = f"Rewrite the following headline to be safe and not contain clickbait:\n\n{headline}"
    rewritten_headline = get_completion([{"role": "user", "content": rewrite_prompt}])
    return rewritten_headline.strip()

# Function to process headlines with CoT
def process_headlines_with_cot(headlines, cot_template):
    cot_responses = []
    for headline in headlines:
        cot_prompt = cot_template.format(headline=headline)
        cot_response = get_completion([{"role": "user", "content": cot_prompt}])
        cot_responses.append(cot_response.strip())
    return cot_responses

# Classify and rewrite headlines based on safety
safe_headlines = []
unsafe_headlines = []
rewritten_unsafe_headlines = {}

for headline in training_headlines + evaluation_headlines:
    moderation_result = moderate_content(headline)
    if any(result["flagged"] for result in moderation_result["results"]):
        unsafe_headlines.append(headline)
        rewritten_headline = rewrite_headline(headline)
        rewritten_unsafe_headlines[headline] = rewritten_headline
    else:
        safe_headlines.append(headline)

# Update the headlines lists with rewritten versions
updated_training_headlines = [rewritten_unsafe_headlines.get(h, h) for h in training_headlines]
updated_evaluation_headlines = [rewritten_unsafe_headlines.get(h, h) for h in evaluation_headlines]

In [5]:
# Function to generate predictions for a list of headlines using a specific template
def generate_predictions(prompt_template, headlines):
    """
    Generates predictions for a list of headlines based on a specified prompt template.

    Args:
        prompt_template (str): The prompt template for generating predictions.
        headlines (list): List of headlines to generate predictions for.

    Returns:
        list: List of predictions for each headline.
    """
    predictions = []
    for headline in headlines:
        prompt = prompt_template.format(headline=headline)
        response = get_completion([{"role": "user", "content": prompt}])
        predictions.append(response.strip())
    return predictions

# Generate zero-shot, few-shot, and CoT predictions
zero_shot_predictions = generate_predictions(zero_shot_template, updated_evaluation_headlines)
few_shot_predictions = generate_predictions(few_shot_template, updated_evaluation_headlines)
cot_responses = process_headlines_with_cot(updated_evaluation_headlines, cot_template)


Chain logged to https://www.comet.com/blaqadonis/my-clickbait-detection-project


INFO:comet_llm.summary:Chain logged to https://www.comet.com/blaqadonis/my-clickbait-detection-project


In [6]:
# Evaluator Prompt
evaluator_prompt = """
You are an evaluator analyzing headlines for clickbait. For each headline, you will be given a prediction stating whether it is 'Clickbait' or 'Legit'. Your task is to assess if this prediction matches the headline's expected label of either Clickbait or Legit.

For each headline and its prediction below, please respond with 'Correct' if the prediction matches the expected label, or 'Incorrect' if it does not.

Headlines and Predictions:
{headline_predictions}

Your evaluation:
"""


In [7]:
# Function to format all predictions for the evaluator
def format_all_predictions_for_evaluator(headlines, predictions, expected_labels):
    """
    Formats all predictions along with their headlines and expected labels for evaluation.

    Args:
        headlines (list): List of headlines.
        predictions (list): List of predictions corresponding to the headlines.
        expected_labels (list): List of expected labels for the headlines.

    Returns:
        str: Formatted string for all predictions.
    """
    formatted_predictions = "\n".join([f"Headline: \"{headline}\", Prediction: {prediction}, Expected: {expected_label}"
                                        for headline, prediction, expected_label in zip(headlines, predictions, expected_labels)])
    return formatted_predictions


def evaluate_prediction(bot_prediction, expected_tag):
    """
    Evaluates a bot's prediction against the expected tag.

    Args:
        bot_prediction (str): The bot's prediction for a headline.
        expected_tag (str): The expected classification for the headline.

    Returns:
        str: 'Correct' if the bot's prediction matches the expected tag, otherwise 'Incorrect'.
    """
    # Simplify the bot's prediction to either 'Clickbait' or 'Legit'
    classification = "Clickbait" if "Clickbait" in bot_prediction else "Legit"

    # Determine if the prediction matches the expected tag
    return "Correct" if classification == expected_tag else "Incorrect"

# Function to evaluate each prediction individually
def evaluate_individual_predictions(evaluator_prompt, headlines, predictions, expected_labels):
    """
    Evaluates each prediction individually using a given evaluator prompt.

    Args:
        evaluator_prompt (str): The evaluator prompt template.
        headlines (list): List of headlines.
        predictions (list): List of predictions for the headlines.
        expected_labels (list): List of expected labels for the headlines.

    Returns:
        list: List of individual evaluation responses for each prediction.
    """
    evaluations = []
    for headline, prediction, expected_label in zip(headlines, predictions, expected_labels):
        formatted_prediction = f"Headline: \"{headline}\", Prediction: {prediction}, Expected: {expected_label}"
        evaluator_response = get_completion([{"role": "user", "content": evaluator_prompt.format(headline_predictions=formatted_prediction)}])
        evaluations.append(evaluator_response.strip())
    return evaluations

# Format all predictions for evaluation (this function is unchanged)
formatted_zero_shot_predictions = format_all_predictions_for_evaluator(evaluation_headlines, zero_shot_predictions, evaluation_labels)
formatted_few_shot_predictions = format_all_predictions_for_evaluator(evaluation_headlines, few_shot_predictions, evaluation_labels)
formatted_cot_predictions = format_all_predictions_for_evaluator(evaluation_headlines, cot_responses, evaluation_labels)

# Evaluate each set of predictions individually
zero_shot_evaluations = evaluate_individual_predictions(evaluator_prompt, evaluation_headlines, zero_shot_predictions, evaluation_labels)
few_shot_evaluations = evaluate_individual_predictions(evaluator_prompt, evaluation_headlines, few_shot_predictions, evaluation_labels)
cot_evaluations = evaluate_individual_predictions(evaluator_prompt, evaluation_headlines, cot_responses, evaluation_labels)


### Logging Evaluations to Comet LLM.

In [8]:
# Enhanced Logging to Comet LLM without prompt template in metadata
for i, headline in enumerate(updated_evaluation_headlines):
    # Ensure indices are within range
    if i >= len(zero_shot_evaluations) or i >= len(few_shot_evaluations) or i >= len(cot_evaluations):
        continue

    # Determine the expected tag for the headline
    original_headline = next((h for h, rh in rewritten_unsafe_headlines.items() if rh == headline), headline)
    expected_tag = evaluation_labels[evaluation_headlines.index(original_headline)]

    # Fetch the evaluation for each type of prediction
    zero_shot_evaluation = zero_shot_evaluations[i]
    few_shot_evaluation = few_shot_evaluations[i]
    cot_evaluation = cot_evaluations[i]

    # Determine if the headline was rewritten for safety
    safety_check = "rewritten" if headline in rewritten_unsafe_headlines.values() else "original"

    # Log zero-shot prediction and evaluation
    comet_llm.log_prompt(
        prompt=zero_shot_template.format(headline=headline),
        output=zero_shot_evaluation,
        tags=["zero-shot", "clickbait-detection"],
        metadata={
            "headline": headline,
            "bot_prediction": zero_shot_predictions[i],
            "expected_tag": expected_tag,
            "evaluation": zero_shot_evaluation,
            "safety_check": safety_check
        },
        api_key=COMET_API_KEY
    )

    # Log few-shot prediction and evaluation
    comet_llm.log_prompt(
        prompt=few_shot_template.format(headline=headline),
        output=few_shot_evaluation,
        tags=["few-shot", "clickbait-detection"],
        metadata={
            "headline": headline,
            "bot_prediction": few_shot_predictions[i],
            "expected_tag": expected_tag,
            "evaluation": few_shot_evaluation,
            "safety_check": safety_check
        },
        api_key=COMET_API_KEY
    )

    # Log CoT prediction and evaluation
    comet_llm.log_prompt(
        prompt=cot_template.format(headline=headline),
        output=cot_evaluation,
        tags=["CoT", "clickbait-detection"],
        metadata={
            "headline": headline,
            "bot_prediction": cot_responses[i],
            "expected_tag": expected_tag,
            "evaluation": cot_evaluation,
            "safety_check": safety_check
        },
        api_key=COMET_API_KEY
    )


### Rewrite

In [9]:
# Function to check for unsafe content based on violence and hate
def is_headline_unsafe(headline):
    moderation_result = moderate_content(headline)
    return any(result['categories'].get('hate') or result['categories'].get('violence')
               for result in moderation_result['results'])

# Function to rewrite headlines and log them
def process_and_log_headlines(dataset_name, headlines):
    for original_headline in headlines:
        if is_headline_unsafe(original_headline):
            rewritten_headline = rewrite_headline(original_headline)
            # Log original and rewritten headline to Comet
            comet_llm.log_prompt(
                prompt="Original Headline: " + original_headline,
                output="Rewritten Headline: " + rewritten_headline,
                tags=["headline-rewriting", "dataset:" + dataset_name],
                metadata={
                    "original_headline": original_headline,
                    "rewritten_headline": rewritten_headline,
                    "dataset": dataset_name
                },
                api_key=COMET_API_KEY
            )

# Process training and evaluation data
process_and_log_headlines("training", training_headlines)
process_and_log_headlines("evaluation", evaluation_headlines)
